<a href="https://colab.research.google.com/github/barabonda/SK-AI-FLY/blob/main/7%EC%A3%BC%EC%B0%A8/LangChain_05_Text_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai langchain tiktoken pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstall


좌표계를 만들어주는 것이 embedding(놓) Model

In [2]:
import os

os.environ['OPENAI_API_KEY'] = '3a9d04966a3c43ca92de7be4b803d6f7'
os.environ['AZURE_OPENAI_ENDPOINT'] = 'https://skyfly-001.openai.azure.com/'
os.environ['OPENAI_API_TYPE'] = 'azure'
os.environ['OPENAI_API_VERSION'] = '2023-05-15'

In [3]:
from langchain.embeddings import AzureOpenAIEmbeddings
embeddings_model = AzureOpenAIEmbeddings(
                        model='dev-text-embedding-ada-002'
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.azure_openai.AzureOpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureOpenAIEmbeddings`.
  warn_deprecated(


In [4]:
embeddings = embeddings_model.embed_documents(
                {
                    '안녕하세요',
                    '제 이름은 홍길동 입니다.',
                    '이름이 무엇인가요?',
                    '랭체인은 유용합니다.',
                    'Hello World'
                }
              )
len(embeddings), len(embeddings[4])
# 임베딩을 거치면 변환해서 엔코딩된다 좌표찍기(임베딩),좌표계의 거리가 1536이다
# 각 데이터는 1536차원                                                        의 배열

(5, 1536)

In [5]:
embedded_query_q = embeddings_model.embed_query('이 대화에 언급된 이름은 무엇입니까?')
embedded_query_a = embeddings_model.embed_query('이 대화에 언급된 이름은 홍길동 입니다.')
print(len(embedded_query_q),len(embedded_query_a))

1536 1536


cosine 맨하튼

In [21]:
from numpy import dot
from numpy.linalg import norm
import numpy as np

def cos_sim(A,B):# 좌표간의 거리
  return dot(A,B)/(norm(A)*norm(B))



In [7]:
print(cos_sin(embedded_query_q, embedded_query_a)) # 높은편
print(cos_sin(embedded_query_q, embeddings[1]))
print(cos_sin(embedded_query_q, embeddings[2]))


0.9085512700834164
0.9216955277777744
0.8496821436212559


# Huggingface Embedding

In [8]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 1.3 MB/s eta 0:00:00


In [9]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = 'BAAI/bge-small-en'
model_kwargs = {'device':'cpu'}
encode_kwargs = {'normalize_embeddings':True}

hf = HuggingFaceBgeEmbeddings(
    model_name = model_name,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
embeddings = hf.embed_documents(
            [
                'today is monday',
                'weather is nice today',
                "what's the problem.",
                'langchain is useful',
                'Hello World',
                'my name is morris'
            ]
        )

In [11]:
BGE_qury_q = hf.embed_query('Hello? who is this?')
BGE_qury_a = hf.embed_query('hi this ;is harrison')

print(cos_sin(BGE_qury_q,BGE_qury_a))
print(cos_sin(BGE_qury_q,embeddings[1]))
print(cos_sin(BGE_qury_q,embeddings[2]))

0.8562599938924814
0.7469068749829496
0.770709978866335


In [12]:
# 0.85 이정도면 꽤 높음

# ko-sbert-nli

In [18]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name= 'jhgan/ko-sbert-nli'
model_kwargs= {'device':'cpu'}
encode_kwargs= {'normalize_embeddings': True}

ko = HuggingFaceBgeEmbeddings(
    model_name = model_name,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs
)

In [19]:
sentence = [
    '안녕하세요',
    '제 이름은 홍길동 입니다.',
    '이름이 무엇인가요?',
    '랭체인은 유용합니다.',
    '홍길동 아버지의 이름은 홍상직입니다.'
]

ko_embeddings = ko.embed_documents(sentence)

In [23]:
q = '홍길동은 아버지를 아버지라 부르지 못했습니다. '
a = '홍길동의 아버지는 엄했습니다.'

ko_query_q = ko.embed_query(q)
ko_query_a = ko.embed_query(a)

print(cos_sim(ko_query_q,ko_query_a))
print(cos_sim(ko_query_q,ko_embeddings[1]))
print(cos_sim(ko_query_q,ko_embeddings[3]))
print(cos_sim(ko_query_q,ko_embeddings[4]))


0.8188146188122829
0.4673269395249799
0.25994464443802445
0.5453010848852629
